# Book Classifier

Projeto desenvolvido para o **Curso de Imersão IA** da [Alura](https://www.alura.com.br/) e Google, explorando o [Gemini](https://gemini.google.com).

O **Gemini** deverá atuar como um bibliotecário, e armazenar cada livro recebido na estante apropriada, aquela que foi pré-definida para armazenar os livros de determinado assunto. Os livros são identificados pelas características que forem fornecidas, por exemplo, Título, Autor, Editora, ISBN, etc.

Então adicionamos uma situação extraordinária, definimos apenas 6 estantes disponíveis com os assuntos relacionados abaixo. Assim, a IA deve tentar reclassificar os assuntos para armazenar o livro na estante correta:

Estante|Assunto
---|---
1|esportes
2|biografia
3|romance
4|culinária
5|terceiro setor
6|outros


Podemos ainda, se necessário, ajustar os critérios de classificação fornecendo novas instruções do tipo ```a editora X só publica livros de romance``` ou ```o autor Y escreve sobre esportes```.

**Importante:** Não esqueça de configurar o ambiente do Colab com sua API_KEY antes de executar este projeto. Veja como fazer isso [aqui](
https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb)


In [70]:
#@title Instalando e configurando a API do Gemini

!pip install -q -U google-generativeai

import google.generativeai as genai
from google.colab import userdata

GOOGLE_API_KEY = userdata.get('gemini_api_secret_name')
genai.configure(api_key=GOOGLE_API_KEY)


In [71]:
#@title Configurando o Google Gemini

#parâmetros de avaliação dos tokens

generation_config = {
  "candidate_count": 1,
  "temperature": 1,
}

#parâmetros de segurança da resposta

safety_settings={
    'HATE': 'BLOCK_NONE',
    'HARASSMENT': 'BLOCK_NONE',
    'SEXUAL' : 'BLOCK_NONE',
    'DANGEROUS' : 'BLOCK_NONE'
    }

# Inicia o modelo
model = genai.GenerativeModel(model_name="gemini-1.0-pro",
                              generation_config=generation_config,
                              safety_settings=safety_settings)

chat = model.start_chat(history=[])

response = chat.send_message(
"""
imagine que você é um bibliotecário e sua biblioteca contém seis estantes, numeradas de 1 a 6,
e em cada estante devem ser armazenados somente os livros do mesmo assunto.

As estantes e o seu respectivo assunto são:
1:esportes
2:biografia
3:romance
4:culinária
5:terceiro setor
6.outros

Então voce precisa ajustar a classificação de cada livro para se encaixar apenas em um destes assuntos,
e com base neste assunto ajustado, identificar em qual estante o livro deve ser armazenado.

Se voce não conseguir identificar o livro, ou não conseguir identificar em qual estante o livro deve ser armazenado, deve armazena-lo na estante 6.

Os livros podem ser identificados por qualquer informação disponível, por exemplo, título, autor, editora, edição, ano da publicação ou pelo código isbn.

Seguem exemplos de classificação:

titulo: o andar do bêbado
editora: zahar
estante: 6

título: Uma terra prometida
autor: barack obama
editora: Companhia das Letras
estante: 2

título: Chef profissional
eitora: senac
estante: 4

título: Simplesmente acontece
Autora: Cecelia Ahern
estante: 3

título: A Pirâmide Invertida
Autor: Jonathan Wilson
estante: 1

título: Cabeça Fria, Coração Quente
Editora: Garoa Livros
estante: 1

Voce vai receber uma lista de livros para serem armazenados, voce deve classificá-los e informar em qual estante ele foi armazenado.
Todas as estantes devem ser esvaziadas agora. A lista de livros deve ser esvaziada agora. Voce só pode armazenar os livros que receber para classinficação a partir de agora.

Quais são as estantes disponíveis para armazenar os novos livros?""")
response.text

'1: Esportes\n2: Biografia\n3: Romance\n4: Culinária\n5: Terceiro Setor\n6: Outros'

### Instruções adicionais

In [78]:
# testando uma maneira alternativa de induzir um novo conhecimento sem treinamento
# se aproveitando do chat, da mesma maneira quando estamos no prompt interativo do Gemini

# alterar a maneira como os resultados são apresentados

response = chat.send_message("""quando receber um novo livro, armazena e informe em qual estante ele foi armazenado, assim:

titulo diario de anne frank
editora record
isbn 8501044458

resposta: Estante 2
""")
print(response.text)

**Título:** Diário de Anne Frank
**Editora:** Record
**ISBN:** 8501044458
**Prateleira:** 2 (Biografia)

**Justificativa:** O livro é um diário escrito por Anne Frank durante a Segunda Guerra Mundial e conta a história de sua família se escondendo dos nazistas. Portanto, o livro pode ser classificado como uma biografia histórica e deve ser armazenado na estante de Biografia.

**Observação:** Embora a Editora Record tenha publicado o livro, a Editora Harlequin não é mencionada como editora. Portanto, a regra de considerar todos os livros da Editora Harlequin como Romance não se aplica neste caso.


In [77]:
# influenciando o algoritmo de classificação da IA

response = chat.send_message("considerar, a partir deste momento, que todos os livros da Editora Harlequin são romance.")
print(response.text)

Entendido. A partir de agora, todos os livros da **Editora Harlequin** serão considerados **Romance** e armazenados na **estante 3**.

Isso significa que, se você receber um livro da Editora Harlequin, ele será automaticamente classificado como Romance e armazenado na estante 3, independentemente do título, autor ou qualquer outra informação.


In [76]:
response = chat.send_message("quais são os livros armazenados na estante 2?")
print(response.text)

Até o momento, os seguintes livros estão armazenados na estante 2:

* **A Segunda Guerra Mundial** de Antony Beevor
* **Diário de Anne Frank** de Anne Frank

Ambos os livros são biografias e, portanto, foram armazenados na estante de Biografia.


### Executando


In [73]:
while True:
  print("Digite informações do livro a ser armazenado, o número da estante para conferir os livros armazenados, ou 0 para sair")
  intext = input()
  if len(intext)==0 or intext=="0":
    break
  elif intext in ["1","2","3","4","5","6"]:
    response = chat.send_message(f"quais são os livros armazenados na estante {intext}?")
    response.text
  else:
    response = chat.send_message(f"armazene o livro: {intext}. em qual estante foi armazenado?")
    print(response.text)

Digite informações do livro a ser armazenado, o número da estante para conferir os livros armazenados, ou 0 para sair
A Segunda Guerra Mundial (Antony Beevor)
**Título:** A Segunda Guerra Mundial
**Autor:** Antony Beevor
**Prateleira:** 2 (Biografia)

**Justificativa:** O livro é uma biografia da Segunda Guerra Mundial, portanto, deve ser armazenado na estante de Biografia.
Digite informações do livro a ser armazenado, o número da estante para conferir os livros armazenados, ou 0 para sair
A Pirâmide Invertida (Jonathan Wilson)
**Título:** A Pirâmide Invertida
**Autor:** Jonathan Wilson
**Prateleira:** 1 (Esportes)

**Justificativa:** O livro é sobre a história tática do futebol, portanto, deve ser armazenado na estante de Esportes.
Digite informações do livro a ser armazenado, o número da estante para conferir os livros armazenados, ou 0 para sair
Simplesmente Acontece (Cecelia Ahern)
**Título:** Simplesmente Acontece
**Autor:** Cecelia Ahern
**Prateleira:** 3 (Romance)

**Justificati

KeyboardInterrupt: Interrupted by user

### Exemplos de livros da minha estante

- A Pirâmide Invertida (Jonathan Wilson)
- Cabeça Fria, Coração Quente (Garoa Livros)
- Lutas de Jiu-Jitsu (Flavio Canto)
- Uma Terra Prometida (Barack Obama)
- A Segunda Guerra Mundial (Antony Beevor)
- A Biografia de Steve Jobs (Walter Isaacson)
- O Menino do Pijama Listrado (John Boyne)
- Simplesmente Acontece (Cecelia Ahern)
- Chef Profissional (Senac)
- Manual da Cozinha Brasileira (Marcelo Fernandes)
- Panelinha: Receitas que Funcionam (Rita Lobo)
- O Poder do Hábito (Charles Duhigg)
- Tudo Começa Com Um Sonho (Alexandre Mattos)
- Tratado de Medicina Interna (Elsevier)
- Manual de Pastoreio de Ovelhas (Agros)
- Introdução à Engenharia de Software (Roger Pressman)
- O Guia do Mochileiro das Galáxias (Douglas Adams)